In this notebook we will present a potential ethical issue. Namely, we will demonstrate an example where slow executions have lower accuracy than fast executions, illustrating that if a model is mostly trained on healthy/uninjured individuals (represented by fast executions), it will fail to predict executions of injured individuals, which are in fact people that would benefit the most from physiotherapy.

In [1]:
import helpers
import numpy as np
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.optim import lr_scheduler
import pandas as pd
import pyarrow
from torchvision import models, transforms
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
# Change paths
model_path = 'paths/ethics.path'
test_path = 'test_set_not_std.parquet'
train_path = 'train_set_not_std_less_noise.parquet'
test_slow_path = 'slow_test.parquet'
test_fast_path = 'fast_test.parquet'

In [4]:
#Importing preprocesssed dataset
test_df = pd.read_parquet(test_path, engine='pyarrow')
train_df = pd.read_parquet(test_path, engine='pyarrow')
test_slow_df = pd.read_parquet(test_slow_path, engine='pyarrow')
test_fast_df = pd.read_parquet(test_fast_path, engine='pyarrow')

Define some dictionnaries

In [3]:
exercise_mapping = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat': 5,
    'Stretch': 6
}
set_mapping = {
    'Correct' : 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
}
camera_mapping = {
    'Frontal_Top': 0,
    'Frontal_Low': 1,
    'Side_Top':2,
    'Side_Low':3

}

In [5]:
video_indices = test_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)
exercise_set_mapping = {exercise_set: index for index, exercise_set in enumerate(train_df[['Exercise', 'Set']].drop_duplicates().itertuples(index=False))}

In [6]:
X_train, X_test, y_train, y_test = helpers.import_data_set(train_df, test_df, set_mapping, False)

In [7]:
# Model for ethical part
get_mlp3x256_set = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(107, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    #nn.Dropout(0.1),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

In [8]:
# Create an instance of the custom dataset
dataset = TensorDataset(X_train, y_train)

# Create a DataLoader
learning_rate = 0.001
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = get_mlp3x256_set()
model.train()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

This is how we trained the model, if you want to retrain, just uncomment, otherwise, directly take the trained path

In [9]:
#for i in range(10):
  #helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)
  #helpers.test_accuracy(model, X_test,y_test,video_indices)

Check accuracy for fast videos

In [10]:
X_train, X_test_fast, y_train, y_test_fast = helpers.import_data_set(train_df, test_fast_df, set_mapping, False)

In [11]:
video_indices = test_fast_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)

In [12]:
helpers.test_accuracy(model, X_test_fast,y_test_fast,video_indices)

Accuracy on each frame: 0.4623
Accuracy for videos: 0.6087
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.73      0.62        73
           1       0.72      0.65      0.69        52
           2       0.61      0.48      0.54        56
           3       0.59      0.48      0.53        21
           4       0.84      0.67      0.74        24
           5       0.00      0.00      0.00         4

    accuracy                           0.61       230
   macro avg       0.55      0.50      0.52       230
weighted avg       0.63      0.61      0.61       230



Check accuracy for slow videos

In [13]:
X_train, X_test_slow, y_train, y_test_slow = helpers.import_data_set(train_df, test_slow_df, set_mapping, False)

In [14]:
video_indices = test_slow_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)

In [15]:
helpers.test_accuracy(model, X_test_slow,y_test_slow,video_indices)

Accuracy on each frame: 0.3109
Accuracy for videos: 0.4105
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.63      0.56        67
           1       0.12      0.21      0.15        24
           2       0.40      0.75      0.52        16
           3       0.70      0.33      0.44        43
           4       0.00      0.00      0.00        16
           5       0.33      0.42      0.37        12
           6       0.00      0.00      0.00        12

    accuracy                           0.41       190
   macro avg       0.29      0.33      0.29       190
weighted avg       0.41      0.41      0.38       190



C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\41794\anaconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
